## Read File Fasta dan buat dict{id:seq}

In [201]:
# from Bio import SeqIO
# record_dict = SeqIO.parse("pengbio.fasta_2", "fasta")
# print(record_dict.id)

dict_new = {}
dict_n = []
index = []

#david fasta 2 memuat 5 sequence sample
from Bio import SeqIO
for record in SeqIO.parse("David.fasta_2", "fasta"):
    index = record.id.split('.')
    dict_new.update({index[1]: str(record.seq)})
#     dict_n.append({index[1]:str(record.seq)})

In [136]:
dict_new #nilai seq

{'1': 'GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCTCGGGTTGTAAACCACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACCGCATATGTGACTGCTTGGCTAGAGTGCGTCAGAGGGAGGTGGAATTCCACGTGTAGCAGTGAAATGCGT',
 '2': 'GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCTCGGGTTGTAAACTACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAATTCCCCGGGCTTAACCTGGGTACCGCATATGTGACTGCTTGGCTAGAGTGCGTCAGAGGGAGGTGGAATTCCACGTGTAGCAGTGAAATGCGT',
 '3': 'GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCGTCGGGTTGTAAACTACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACCGCATATGTGACTGCT

### menghitung rata rata  panjang seq

In [139]:
def meanLength(data): 
    s = 0
    tot = 0
    for x in data:
        s += len(data[x])
        tot += 1
    return s/float(tot)

In [140]:
meanLength(dict_new)

341.8

## fungsi mencari overlap antar 2 seq

In [141]:
def getOverlap(left, right):
    for i in range(len(left)):
        if left[i:] == right[:len(left)-i]:
            return left[i:]
    return ''

## mencari overlaps antar seq

In [143]:
def getAllOverlaps(reads):
    d = dict()
    for name1, seq1 in reads.items():
        for name2, seq2 in reads.items():
            if name1 == name2:
                continue
            if name1 not in d:
                d[name1] = dict()
            d[name1][name2] = len(getOverlap(seq1, seq2))

    return d

In [180]:
d=getAllOverlaps(dict_new)
d

{'1': {'2': 0, '3': 0, '4': 0, '5': 0},
 '2': {'1': 0, '3': 0, '4': 0, '5': 0},
 '3': {'1': 1, '2': 1, '4': 1, '5': 1},
 '4': {'1': 0, '2': 0, '3': 0, '5': 0},
 '5': {'1': 0, '2': 0, '3': 0, '4': 0}}

In [181]:
def prettyPrint(d):
    print (' D  '),
    for j in sorted(d, key=int):
        print("(% 3s)" % j), #biar spasi panjang
    print()

    for i in sorted(d, key=int):
        print ("(% 3s)" % i),
        for j in sorted(d, key=int):
            if i == j:
                s = '  -'
            else:
                s = "(% 3s)" % d[str(i)][str(j)]
            print(s),
        print()

In [182]:
#prettyPrint(d)

In [183]:
def findFirstRead(overlaps):
    for i in overlaps:
        print(i)
        signifOverlaps = False
        for j in d[i]:
            if d[j][i] > 3:
                signifOverlaps = True
        if not signifOverlaps:
            return i

In [184]:
findFirstRead(d)

1


'1'

In [202]:
def findKeyForLargestValue(d):
#     return sorted(d.items(), key=lambda x: x[1])[-1][0]
    m = max(d.values())
    for k in d:
        if d[k] == m:
            return k

In [188]:
def findOrder(first, d):
    if max(d[first].values()) < 3:
        return [first]
    else:
        nextRead = findKeyForLargestValue(d[first])
        return [first] + findOrder(nextRead, d)

In [197]:
order=findOrder(findFirstRead(dict_new), getAllOverlaps(dict_new))

1


In [198]:
def assembleGenome(readOrder, reads, overlaps):
    genome = ''
    for readName in readOrder[:-1]:
        rightOverlap = max(x for x in overlaps[readName].values() if x >= 3)
        genome += reads[readName][:-rightOverlap]
    genome += reads[readOrder[-1]]
    return genome

In [205]:
reference = assembleGenome(order,dict_new,d)
print (reference)
print ("Reference gen terpanjang",len(reference))


GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCTCGGGTTGTAAACCACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACCGCATATGTGACTGCTTGGCTAGAGTGCGTCAGAGGGAGGTGGAATTCCACGTGTAGCAGTGAAATGCGT
Reference gen terpanjang 342
